# Training Experiments

This notebook contains various training experiments for the Stock Movement Prediction System.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv('../data/stock_data.csv', parse_dates=['Date'])

# Preprocess the data
data['PriceChange'] = data['Close'].pct_change()
data = data.dropna()
X = data[['Open', 'High', 'Low', 'Volume']].values
y = data['Close'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_train_scaled = scaler.fit_transform(y_train.reshape(-1, 1))
y_test_scaled = scaler.transform(y_test.reshape(-1, 1))

# Build the model
model = models.Sequential([
    layers.InputLayer(input_shape=(X_train_scaled.shape[1],)),
    layers.LSTM(64, return_sequences=True),
    layers.LSTM(64),
    layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train_scaled, y_train_scaled, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test_scaled))

# Plot training history
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()